In [1]:
import hashlib
import binascii

In computer programming hash functions map text (or other data) to integer numbers. Usually different inputs maps to different outputs, but sometimes a collision may happen (different input with the same output).
Cryptographic hash functions transform text or binary data to fixed-length hash value and are known to be collision-resistant and irreversible. Example of cryptographic hash function is SHA3-256:

In [36]:
sha3_512hash = hashlib.sha3_512(b'hello')
print(sha3_512hash.digest_size)
print("SHA3-512('hello') =", binascii.hexlify(sha3_512hash.digest()))

64
SHA3-512('hello') = b'75d527c368f2efe848ecf6b073a36767800805e9eef2b1857d5f984f036eb6df891d75f72d9b154518c1cd58835286d1da9a38deba3de98b5a53e5ed78a84976'


In [37]:
import hmac

In [38]:
key = b"12345"
msg = b"sample message"
hmachash = hmac.new(key, msg, hashlib.sha3_512).hexdigest()
print(hmachash, len(hmachash))

1738802bb9ea2be407538f6a7711b86310f16628d3f95cbb04056b659a389ac37fb0c086dd3c4415442e0bf27e3107a4476990b11cf54ee73ea87eb014a2dd8d 128


In [39]:
import os
password = b'haslo'
salt = os.urandom(128)
iterations = 100_000
hashed = hashlib.pbkdf2_hmac("sha3-512", password, salt, iterations)
print(hashed.hex())
print(len(hashed.hex()))

128
70d29a304ad3c99b124e47d02e15f9c27f9ac1f14b20bad51b3dfbda1cfddfe9ee104e9714c66c18379bf6d695e798beb78bcd6fa43914810ef5640580449dd9
128


# Scrypt
N – iterations count (affects memory and CPU usage), e.g. 16384 or 2048

r – block size (affects memory and CPU usage), e.g. 8

p – parallelism factor (threads to run in parallel - affects the memory, CPU usage), usually 1

password– the input password (8-10 chars minimal length is recommended)

salt – securely-generated random bytes (64 bits minimum, 128 bits recommended)

derived-key-length - how many bytes to generate as output, e.g. 32 bytes (256 bits)

In [148]:
password = b'haslo'
salt = os.urandom(16)
key = hashlib.scrypt(password=password, salt=salt, n=2**14, r=8, p=1)
print(key.hex())

00853ae8eafbe47ee451553335e42ec336565450d8e912d5154d258b99f975f27ddbb5d9943b2c4f7526ea642b88f4d206527586f1accacf433e8d5efff0bcc1


## Memory required = 128 * N * r * p bytes

In [91]:
memory = 128 * 2**14 * 8 * 1
print(memory/1024/1024, "MB")

16.0 MB


# How passwords should be stored in databases

In [149]:
from base64 import b64encode, b64decode
def genHash(password: bytes, salt: bytes, n: int, r: int, p: int):
    key = hashlib.scrypt(password=password, salt=salt, n=n, r=r, p=p)
    return f"{n}${r}${p}${salt.hex()}${key.hex()}"

password = b'haslo'
salt = os.urandom(32)
# print(genHash(password, salt, 2**14, 8, 1))
#16384$8$1$kytG1MHY1KU=$afc338d494dc89be40e317788e3cd9166d066709db0e6481f0801bd918710f46

In [16]:
import argon2, binascii

hash = argon2.hash_password_raw(
    time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32,
    password=b'password', salt=b'some salt', type=argon2.low_level.Type.ID)
print("Argon2 raw hash:", binascii.hexlify(hash))

argon2Hasher = argon2.PasswordHasher(
    time_cost=2, memory_cost=2**15, parallelism=2, hash_len=32, salt_len=16)
hash = argon2Hasher.hash("password")
print("Argon2 hash (random salt):", hash)

verifyValid = argon2Hasher.verify(hash, "password")
print("Argon2 verify (correct password):", verifyValid)

try:
    argon2Hasher.verify(hash, "wrong123")
except:
    print("Argon2 verify (incorrect password):", False)

Argon2 raw hash: b'157f21dd3fdf7bafb76d2923ccaffa0b7be7cbae394709474d2bc66ee7b09d3e'
Argon2 hash (random salt): $argon2id$v=19$m=32768,t=2,p=2$Jx6BX/4ZR3Z+EkKnoHb/ZA$zDAFZgL5MdeKSrvDMSMzFZST1zlNodVm306pVRzrg9I
Argon2 verify (correct password): True
Argon2 verify (incorrect password): False


# Recommended 4GB for backend, 1GB for front-end

In [30]:
%%time
argon2Hasher = argon2.PasswordHasher(memory_cost=1024*1024*1)
backendHash = argon2Hasher.hash("haslo")
print(backendHash)

$argon2id$v=19$m=1048576,t=2,p=8$Tkew9mn0kOZDRgyX+sYOmg$DPyi0+Mtg9pG7bT1aG5OPQ
CPU times: user 2.47 s, sys: 488 ms, total: 2.96 s
Wall time: 477 ms


In [32]:
if argon2Hasher.check_needs_rehash("$argon2id$v=18$m=1048576,t=2,p=8$onuHP4BZEXp+cJAly0f+7A$Q51bPkzVUSsQ+jP5GMd7Wg"):
    print(argon2Hasher.hash("podane_haslo"))

$argon2id$v=19$m=1048576,t=2,p=8$SHS7xEUxBES2IMySIPsNFQ$AtVtDfUTboxIvOhv9yE+lA


In [22]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_CTR)
ciphertext, tag = cipher.encrypt_and_digest(b"eluwina")

file_out = open("encrypted.bin", "wb")
[ file_out.write(x) for x in (cipher.nonce, tag, ciphertext) ]
file_out.close()

In [23]:
file_in = open("encrypted.bin", "rb")
nonce, tag, ciphertext = [ file_in.read(x) for x in (16, 16, -1) ]

# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(ciphertext, tag)
print(data)

b'eluwina'
